# Assignment 5: Visual Servoing

The following has ALL the code to run the simulation and all the problems. You do not need to understand exactly how it works. You only need to work on the sections starting at "COMPLETE THIS". 

In [ ]:
#import matplotlib.pyplot as plt
import numpy as np
import pybullet as p
import pybullet_data
import os    
import numpy as np
import time
import cv2
from scipy.spatial.transform import Rotation as Rot            #can use this to apply angular rotations to coordinate frames

#camera (don't change these settings)
camera_width = 512                                             #image width
camera_height = 512                                            #image height
camera_fov = 120                                                #field of view of camera
camera_focal_depth = 0.5*camera_height/np.tan(0.5*np.pi/180*camera_fov) 
                                                               #focal depth in pixel space
camera_aspect = camera_width/camera_height                     #aspect ratio
camera_near = 0.02                                             #near clipping plane in meters, do not set non-zero
camera_far = 100                                               #far clipping plane in meters


#control objectives (if you wish, you can play with these values for fun)
object_location_desired = np.array([camera_width/2,camera_height/2])
                                                               #center the object to middle of image 
K_p_x = 0.1                                                    #Proportional control gain for translation
K_p_Omega = 0.02                                               #Proportional control gain for rotation       

### Create the Robot Class

In [2]:
# Robot with Camera Class
class eye_in_hand_robot:
    def get_ee_position(self):
        '''
        Function to return the end-effector of the link. This is the very tip of the robot at the end of the jaws.
        '''
        endEffectorIndex = self.numActiveJoints
        endEffectorState = p.getLinkState(self.robot_id, endEffectorIndex)
        endEffectorPos = np.array(endEffectorState[0])
        endEffectorOrn = np.array(p.getMatrixFromQuaternion(endEffectorState[1])).reshape(3,3)
        
        #add an offset to get past the forceps
        endEffectorPos += self.camera_offset*endEffectorOrn[:,2]
        return endEffectorPos, endEffectorOrn

    def get_current_joint_angles(self):
        # Get the current joint angles
        joint_angles = np.zeros(self.numActiveJoints)
        for i in range(self.numActiveJoints):
            joint_state = p.getJointState(self.robot_id, self._active_joint_indices[i])
            joint_angles[i] = joint_state[0]
        return joint_angles
    
    def get_jacobian_at_current_position(self):
        #Returns the Robot Jacobian of the last active link
        mpos, mvel, mtorq = self.get_active_joint_states()   
        zero_vec = [0.0]*len(mpos)
        linearJacobian, angularJacobian = p.calculateJacobian(self.robot_id, 
                                                              self.numActiveJoints,
                                                              [0,0,self.camera_offset],
                                                              mpos, 
                                                              zero_vec,
                                                              zero_vec)
        #only return the active joint's jacobians
        Jacobian = np.vstack((linearJacobian,angularJacobian))
        return Jacobian[:,:self.numActiveJoints]
    
    def set_joint_position(self, desireJointPositions, kp=1.0, kv=0.3):
        '''Set  the joint angle positions of the robot'''
        zero_vec = [0.0] * self._numLinkJoints
        allJointPositionObjectives = [0.0]*self._numLinkJoints
        for i in range(desireJointPositions.shape[0]):
            idx = self._active_joint_indices[i]
            allJointPositionObjectives[idx] = desireJointPositions[i]

        p.setJointMotorControlArray(self.robot_id,
                                    range(self._numLinkJoints),
                                    p.POSITION_CONTROL,
                                    targetPositions=allJointPositionObjectives,
                                    targetVelocities=zero_vec,
                                    positionGains=[kp] * self._numLinkJoints,
                                    velocityGains=[kv] * self._numLinkJoints)

    def get_active_joint_states(self):
        '''Get the states (position, velocity, and torques) of the active joints of the robot
        '''
        joint_states = p.getJointStates(self.robot_id, range(self._numLinkJoints))
        joint_infos = [p.getJointInfo(self.robot_id, i) for i in range(self._numLinkJoints)]
        joint_states = [j for j, i in zip(joint_states, joint_infos) if i[3] > -1]
        joint_positions = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        joint_torques = [state[3] for state in joint_states]
        return joint_positions, joint_velocities, joint_torques


         
    def __init__(self, robot_id, initialJointPos):
        self.robot_id = robot_id
        self.eeFrameId = []
        self.camera_offset = 0.1 #offset camera in z direction to avoid grippers
        # Get the joint info
        self._numLinkJoints = p.getNumJoints(self.robot_id) #includes passive joint
        jointInfo = [p.getJointInfo(self.robot_id, i) for i in range(self._numLinkJoints)]
        
        # Get joint locations (some joints are passive)
        self._active_joint_indices = []
        for i in range(self._numLinkJoints):
            if jointInfo[i][2]==p.JOINT_REVOLUTE:
                self._active_joint_indices.append(jointInfo[i][0])
        self.numActiveJoints = len(self._active_joint_indices) #exact number of active joints

        #reset joints
        for i in range(self._numLinkJoints):
            p.resetJointState(self.robot_id,i,initialJointPos[i])

### Ancilliary Functions

In [3]:
def draw_coordinate_frame(position, orientation, length, frameId = []):
    '''
    Draws a coordinate frame x,y,z with scaled lengths on the axes 
    in a position and orientation relative to the world coordinate frame
    pos: 3-element numpy array
    orientation: 3x3 numpy matrix
    length: length of the plotted x,y,z axes
    frameId: a unique ID for the frame. If this supplied, then it will erase the previous location of the frame
    
    returns the frameId
    '''
    if len(frameId)!=0:
        p.removeUserDebugItem(frameId[0])
        p.removeUserDebugItem(frameId[1])
        p.removeUserDebugItem(frameId[2])
    
    lineIdx=p.addUserDebugLine(position, position + np.dot(orientation, [length, 0, 0]), [1, 0, 0])  # x-axis in red
    lineIdy=p.addUserDebugLine(position, position + np.dot(orientation, [0, length, 0]), [0, 1, 0])  # y-axis in green
    lineIdz=p.addUserDebugLine(position, position + np.dot(orientation, [0, 0, length]), [0, 0, 1])  # z-axis in blue

    return lineIdx,lineIdy,lineIdz

def opengl_plot_world_to_pixelspace(pt_in_3D_to_project, viewMat, projMat, imgWidth, imgHeight):
    ''' Plots a x,y,z location in the world in an openCV image
    This is used for debugging, e.g. given a known location in the world, verify it appears in the camera
    when using p.getCameraImage(...). The output [u,v], when plot with opencv, should line up with object 
    in the image from p.getCameraImage(...)
    '''
    pt_in_3D_to_project = np.append(pt_in_3D_to_project,1)
    #print('Point in 3D to project:', pt_in_3D_to_project)

    pt_in_3D_in_camera_frame = viewMat @ pt_in_3D_to_project
    #print('Point in camera space: ', pt_in_3D_in_camera_frame)

    # Convert coordinates to get normalized device coordinates (before rescale)
    uvzw = projMat @ pt_in_3D_in_camera_frame
    #print('after projection: ', uvzw)

    # scale to get the normalized device coordinates
    uvzw_NDC = uvzw/uvzw[3]
    #print('after normalization: ', uvzw_NDC)

    #x,y specifies lower left corner of viewport rectangle, in pixels. initial value is (0,)
    u = ((uvzw_NDC[0] + 1) / 2.0) * imgWidth
    v = ((1-uvzw_NDC[1]) / 2.0) * imgHeight

    return [int(u),int(v)]

    
def get_camera_view_and_projection_opencv(cameraPos, camereaOrn):
    '''Gets the view and projection matrix for a camera at position (3) and orientation (3x3)'''
    __camera_view_matrix_opengl = p.computeViewMatrix(cameraEyePosition=cameraPos,
                                                   cameraTargetPosition=cameraPos+camereaOrn[:,2],
                                                   cameraUpVector=-camereaOrn[:,1])

    __camera_projection_matrix_opengl = p.computeProjectionMatrixFOV(camera_fov, camera_aspect, camera_near, camera_far)        
    _, _, rgbImg, depthImg, _ = p.getCameraImage(camera_width, 
                                                 camera_height, 
                                                 __camera_view_matrix_opengl,
                                                 __camera_projection_matrix_opengl, 
                                                 renderer=p.ER_BULLET_HARDWARE_OPENGL)

    #returns camera view and projection matrices in a form that fits openCV
    viewMat = np.array(__camera_view_matrix_opengl).reshape(4,4).T
    projMat = np.array(__camera_projection_matrix_opengl).reshape(4,4).T
    return viewMat, projMat

def get_camera_img_float(cameraPos, camereaOrn):
    ''' Gets the image and depth map from a camera at a position cameraPos (3) and cameraOrn (3x3) in space. '''
    __camera_view_matrix_opengl = p.computeViewMatrix(cameraEyePosition=cameraPos,
                                                   cameraTargetPosition=cameraPos+camereaOrn[:,2],
                                                   cameraUpVector=-camereaOrn[:,1])

    __camera_projection_matrix_opengl = p.computeProjectionMatrixFOV(camera_fov, camera_aspect, camera_near, camera_far)        
    width, height, rgbImg, nonlinDepthImg, _ = p.getCameraImage(camera_width, 
                                                 camera_height, 
                                                 __camera_view_matrix_opengl,
                                                 __camera_projection_matrix_opengl, 
                                                 renderer=p.ER_BULLET_HARDWARE_OPENGL)

    #adjust for clipping and nonlinear distance i.e., 1/d (0 is closest, i.e., near, 1 is furthest away, i.e., far
    depthImgLinearized =camera_far*camera_near/(camera_far+camera_near-(camera_far-camera_near)*nonlinDepthImg)

    #convert to numpy and a rgb-d image
    rgb_image = np.array(rgbImg[:,:,:3], dtype=np.uint8)
    depth_image = np.array(depthImgLinearized, dtype=np.float32)
    return rgb_image, depth_image


### Create Physics Environment

In [4]:
# Start the connection to the physics server
physicsClient = p.connect(p.GUI)#(p.DIRECT)
time_step = 0.001
p.resetSimulation()
p.setTimeStep(time_step)
p.setGravity(0, 0, -9.8)

# Set the path to the URDF files included with PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# load a plane URDF
p.loadURDF('plane.urdf')

# Place obstacles
box_length = box_width = 0.4
box_depth = 0.02
object_center = [0.3, 1, 0.01]
object_orientation = [0, 0, 0]
object_color = [0.8, 0.0, 0.0, 1]
geomBox = p.createCollisionShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2])
visualBox = p.createVisualShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2], rgbaColor=object_color)
boxId = p.createMultiBody(
    baseMass=0,
    baseCollisionShapeIndex=geomBox,
    baseVisualShapeIndex=visualBox,
    basePosition=np.array(object_center),
    baseOrientation=p.getQuaternionFromEuler(object_orientation)
)
ObjectModelPos, modelOrn  = p.getBasePositionAndOrientation(boxId)

#reset debug gui camera position so we can see the robot up close
p.resetDebugVisualizerCamera( cameraDistance=1, cameraYaw=30, cameraPitch=-52, cameraTargetPosition=[0,0,.5])
p.disconnect()

### COMPLETE THIS

In [5]:
## Q1: Show movement in Jacobian
def getImageJacobian(u_px,v_px,depthImg,focal_length, imgWidth, imgHeight):
    ''' Inputs: 
    u_px, v_px is pixel coordinates from image with top-left corner as 0,0
    depthImg is the depth map
    f is the focal length
    
    Outputs: image_jacobian, a 2x6 matrix'''


    # Convert to image-centered coordinates
    u = u_px - imgWidth / 2.0
    v = imgHeight / 2.0 - v_px   # invert y so +v is up
    
    # Get depth at pixel
    Z = depthImg[int(v_px), int(u_px)]
    
    f = focal_length
    image_jacobian = np.zeros((2,6))
    
    # Fill in entries
    image_jacobian[0,0] = -f / Z
    image_jacobian[0,1] = 0
    image_jacobian[0,2] = u / Z
    image_jacobian[0,3] = (u * v) / f
    image_jacobian[0,4] = -(f**2 + u**2) / f
    image_jacobian[0,5] = v
    
    image_jacobian[1,0] = 0
    image_jacobian[1,1] = -f / Z
    image_jacobian[1,2] = v / Z
    image_jacobian[1,3] = (f**2 + v**2) / f
    image_jacobian[1,4] = -(u * v) / f
    image_jacobian[1,5] = -u


    return image_jacobian
    

In [6]:
def findCameraControl(object_loc_des, object_loc, image_jacobian):
    ''' Inputs:
    object_loc_des: desired [x,y] pixel locations for object
    object_loc: current [x,y] pixel locations as found from computer vision
    image_jacobian: the image jacobian 
        Outputs:
    delta_X: the scaled displacement in position of camera (world frame) to reduce the error
    delta_Omega: the scaled angular velocity of camera (world frame omega-x,y,z) to reduce the error
    
    '''
    lam = 0.5
    # Compute image error (2x1)
    e = np.array(object_loc) - np.array(object_loc_des)   # [u - u*, v - v*]
    
    # Compute pseudoinverse of image Jacobian (6x2)
    L_pinv = np.linalg.pinv(image_jacobian)
    
    # Compute camera twist (6x1)
    v_c = -lam * L_pinv @ e     # control law
    
    # Split into translation and rotation
    delta_X = v_c[0:3]          # translational velocity
    delta_Omega = v_c[3:6]      # angular velocity
    
    return delta_X, delta_Omega
    
    # return delta_X = [], delta_Omega 

In [ ]:

#     return new_jointPositions = []


def findJointControl(robot, delta_X, delta_Omega):
    ''' Inputs:
    delta_X: the scaled displacement in position of camera (world frame) to reduce the error
    delta_Omega: the scaled angular velocity of camera (world frame omega-x,y,z) to reduce the error
    Outputs:
    delta_Q: the change in robot joints to cause the camera to move delta_X and delta_Omega
    '''

    # Combine translational and rotational components into a single 6x1 twist
    v_c = np.hstack((delta_X, delta_Omega))  # [vx, vy, vz, wx, wy, wz]
    
    # Get current joint angles
    q_current = np.array(robot.get_current_joint_angles())
    
    # Get current geometric Jacobian (6 x n)
    J = robot.get_jacobian_at_current_position()
    
    # Compute pseudoinverse of Jacobian
    J_pinv = np.linalg.pinv(J)
    
    # Compute joint velocity command (change in joint space)
    delta_Q = J_pinv @ v_c    # (n,)
    
    # Compute new joint positions
    new_jointPositions = q_current + delta_Q

    return new_jointPositions


## SIMULATE ANSWERS

### Q1 and Q2 Setup

In [12]:
import pybullet as p
import pybullet_data
import numpy as np
import cv2
import imageio
import os

# ------------------- PHYSICS SETUP -------------------
physicsClient = p.connect(p.GUI)
time_step = 0.001
p.resetSimulation()
p.setTimeStep(time_step)
p.setGravity(0, 0, -9.8)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load environment
p.loadURDF('plane.urdf')

# Place red box (target object)
box_length = box_width = 0.4
box_depth = 0.02
object_center = [0.3, 1, 0.01]
object_orientation = [0, 0, 0]
object_color = [0.8, 0.0, 0.0, 1]
geomBox = p.createCollisionShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2])
visualBox = p.createVisualShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2],
                                rgbaColor=object_color)
boxId = p.createMultiBody(
    baseMass=0,
    baseCollisionShapeIndex=geomBox,
    baseVisualShapeIndex=visualBox,
    basePosition=np.array(object_center),
    baseOrientation=p.getQuaternionFromEuler(object_orientation)
)

# Camera visualization setup
p.resetDebugVisualizerCamera(cameraDistance=1, cameraYaw=30, cameraPitch=-52, cameraTargetPosition=[0, 0, 0.5])

# ------------------- CAMERA PARAMETERS -------------------
camera_width = 512
camera_height = 512
camera_focal_depth = 1.0
motion_distance = 0.1       # meters for significant motion
rotation_angle = np.deg2rad(15)  # radians
num_steps = 40              # smooth frames per motion
fps = 5

# Initial camera pose
camera_init_pos = np.array([0.2, 0, 1])
camera_init_rot = np.array([[1, 0, 0],
                            [0, -1, 0],
                            [0, 0, -1]])

# Define motions (6 DOFs)
motions = {
    "No_Motion": {"delta_X": np.array([0, 0, 0]), "delta_Omega": np.zeros(3)},
    "Trans_X+": {"delta_X": np.array([motion_distance, 0, 0]), "delta_Omega": np.zeros(3)},
    "Trans_Y+": {"delta_X": np.array([0, motion_distance, 0]), "delta_Omega": np.zeros(3)},
    "Trans_Z+": {"delta_X": np.array([0, 0, motion_distance]), "delta_Omega": np.zeros(3)},
    "Rot_X+":   {"delta_X": np.zeros(3), "delta_Omega": np.array([rotation_angle, 0, 0])},
    "Rot_Y+":   {"delta_X": np.zeros(3), "delta_Omega": np.array([0, rotation_angle, 0])},
    "Rot_Z+":   {"delta_X": np.zeros(3), "delta_Omega": np.array([0, 0, rotation_angle])},
}

# ------------------- RECORD EACH MOTION -------------------
for label, motion in motions.items():
    print(f"\n=== Recording significant motion: {label} ===")
    frames = []

    # Reset camera to initial pose
    cameraPosition = camera_init_pos.copy()
    cameraOrientation = camera_init_rot.copy()

    for step in range(num_steps):
        # Apply smooth motion
        alpha = step / (num_steps - 1)
        delta_X = motion["delta_X"] * alpha
        delta_Omega = motion["delta_Omega"] * alpha

        # Translation update
        cameraPosition = camera_init_pos + delta_X

        # Rotation update using small-angle approximation
        omega_skew = np.array([
            [0, -delta_Omega[2], delta_Omega[1]],
            [delta_Omega[2], 0, -delta_Omega[0]],
            [-delta_Omega[1], delta_Omega[0], 0]
        ])
        delta_R = np.eye(3) + omega_skew
        cameraOrientation = camera_init_rot @ delta_R

        # Re-orthonormalize rotation matrix
        U, _, Vt = np.linalg.svd(cameraOrientation)
        cameraOrientation = U @ Vt

        # --- Get camera image ---
        rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)

        # --- Get object pixel coordinates ---
        viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
        object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,
                                                     camera_width, camera_height)

        # --- Draw object location for visualization ---
        img_disp = (rgb * 255).astype(np.uint8)
        u, v = int(object_loc[0]), int(object_loc[1])
        if 0 <= u < camera_width and 0 <= v < camera_height:
            cv2.circle(img_disp, (u, v), 8, (0, 255, 0), -1)
        cv2.putText(img_disp, f"{label} step {step}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        frames.append(img_disp)
        p.stepSimulation()

    # Save motion GIF
    gif_name = f"{label}_motion.gif"
    imageio.mimsave(gif_name, frames, fps=fps)
    print(f"Saved {gif_name}")

p.disconnect()



=== Recording significant motion: No_Motion ===
Saved No_Motion_motion.gif

=== Recording significant motion: Trans_X+ ===
Saved Trans_X+_motion.gif

=== Recording significant motion: Trans_Y+ ===
Saved Trans_Y+_motion.gif

=== Recording significant motion: Trans_Z+ ===
Saved Trans_Z+_motion.gif

=== Recording significant motion: Rot_X+ ===
Saved Rot_X+_motion.gif

=== Recording significant motion: Rot_Y+ ===
Saved Rot_Y+_motion.gif

=== Recording significant motion: Rot_Z+ ===
Saved Rot_Z+_motion.gif


In [8]:
import imageio

# Start the connection to the physics server
physicsClient = p.connect(p.GUI)
time_step = 0.001
p.resetSimulation()
p.setTimeStep(time_step)
p.setGravity(0, 0, -9.8)

# Set the path to the URDF files included with PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# load a plane URDF
p.loadURDF('plane.urdf')

# Place obstacles
box_length = box_width = 0.4
box_depth = 0.02
object_center = [0.3, 1, 0.01]
object_orientation = [0, 0, 0]
object_color = [0.8, 0.0, 0.0, 1]
geomBox = p.createCollisionShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2])
visualBox = p.createVisualShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2], rgbaColor=object_color)
boxId = p.createMultiBody(
    baseMass=0,
    baseCollisionShapeIndex=geomBox,
    baseVisualShapeIndex=visualBox,
    basePosition=np.array(object_center),
    baseOrientation=p.getQuaternionFromEuler(object_orientation)
)
ObjectModelPos, modelOrn  = p.getBasePositionAndOrientation(boxId)

#reset debug gui camera position so we can see the robot up close
p.resetDebugVisualizerCamera( cameraDistance=1, cameraYaw=30, cameraPitch=-52, cameraTargetPosition=[0,0,.5])

cameraPosition = np.array([0.2,0,1])
cameraOrientation = np.array([[1,0,0],[0,-1,0],[0,0,-1]])





# Camera and image parameters
camera_width = 512
camera_height = 512
camera_focal_depth = 1.0  # approximate focal length (adjust to your getImageJacobian convention)
motion_delta = 0.02        # small translational delta (m)
angle_delta = np.deg2rad(5)  # small angular delta (radians)
num_steps = 100             # number of intermediate frames for smooth GIF
fps = 5

# Initial camera pose
camera_init_pos = np.array([0.2, 0, 1])
camera_init_rot = np.array([[1,0,0],[0,-1,0],[0,0,-1]])

# 6-axis motion labels and corresponding deltas
motions = {
    "Trans_X+": {"delta_X": np.array([motion_delta, 0, 0]), "delta_Omega": np.zeros(3)},
    "Trans_Y+": {"delta_X": np.array([0, motion_delta, 0]), "delta_Omega": np.zeros(3)},
    "Trans_Z+": {"delta_X": np.array([0, 0, motion_delta]), "delta_Omega": np.zeros(3)},
    "Rot_X+":   {"delta_X": np.zeros(3), "delta_Omega": np.array([angle_delta, 0, 0])},
    "Rot_Y+":   {"delta_X": np.zeros(3), "delta_Omega": np.array([0, angle_delta, 0])},
    "Rot_Z+":   {"delta_X": np.zeros(3), "delta_Omega": np.array([0, 0, angle_delta])},
}

for label, motion in motions.items():
    print(f"\n=== Running motion test: {label} ===")
    frames = []

    for step in range(num_steps):
        # Reset to initial pose each time to avoid drift
        cameraPosition = camera_init_pos.copy()
        cameraOrientation = camera_init_rot.copy()

        # Apply proportional scaling of motion by step
        delta_scale = (step / (num_steps - 1))
        delta_X = motion["delta_X"] * delta_scale
        delta_Omega = motion["delta_Omega"] * delta_scale

        # Apply translation
        cameraPosition = camera_init_pos + delta_X

        # Apply rotation using small-angle approximation
        omega_skew = np.array([[0, -delta_Omega[2], delta_Omega[1]],
                               [delta_Omega[2], 0, -delta_Omega[0]],
                               [-delta_Omega[1], delta_Omega[0], 0]])
        delta_R = np.eye(3) + omega_skew
        cameraOrientation = camera_init_rot @ delta_R
        U, _, Vt = np.linalg.svd(cameraOrientation)
        cameraOrientation = U @ Vt  # re-orthonormalize

        # Render image
        rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)

        # Get pixel coordinates of the object
        viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
        object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,
                                                     camera_width, camera_height)

        # Draw object marker for visualization
        img_disp = rgb.copy()
        u, v = int(object_loc[0]), int(object_loc[1])
        cv2.circle(img_disp, (u, v), 8, (0, 255, 0), -1)
        cv2.putText(img_disp, f"{label} step {step}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        frames.append((img_disp * 255).astype(np.uint8))

    # Save GIF
    gif_name = f"{label}_motion.gif"
    imageio.mimsave(gif_name, frames, fps=fps)
    print(f"Saved: {gif_name}")

p.disconnect()

# for ITER in range(200):
#     p.stepSimulation()
 
#     ''' Get Image'''
#     rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
        
#     ''' Magical Computer Vision Algorithm that gets locations of objects in the image, as object_loc (Do Not Remove)'''
#     viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
#     object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,camera_width, camera_height)
    
#     ''' Get Image Jacobian '''
#     imageJacobian = getImageJacobian(object_loc[0], object_loc[1], depth, camera_focal_depth, camera_width, camera_height)

#     ''' Get Camera Control '''
#     delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, imageJacobian)
    
#     ''' Apply Camera Controls'''
#     # Update camera position using proportional control
#     cameraPosition = cameraPosition + K_p_x * delta_X
    
#     # Update camera orientation using proportional control for angular velocity
#     # Convert angular velocity to rotation matrix update
#     # For small rotations, we can approximate: R_new ≈ R * (I + [omega]_×)
#     omega_skew = np.array([[0, -delta_Omega[2], delta_Omega[1]],
#                            [delta_Omega[2], 0, -delta_Omega[0]],
#                            [-delta_Omega[1], delta_Omega[0], 0]])
#     delta_R = np.eye(3) + K_p_Omega * omega_skew
#     cameraOrientation = cameraOrientation @ delta_R
#     # Re-orthonormalize to ensure it remains a rotation matrix
#     U, _, Vt = np.linalg.svd(cameraOrientation)
#     cameraOrientation = U @ Vt

#     # show image
#     cv2.imshow("depth", depth)
#     cv2.imshow("rgb", rgb)
#     cv2.waitKey(1)

# #close the physics server
# cv2.destroyAllWindows()    
# p.disconnect() 



=== Running motion test: Trans_X+ ===
Saved: Trans_X+_motion.gif

=== Running motion test: Trans_Y+ ===
Saved: Trans_Y+_motion.gif

=== Running motion test: Trans_Z+ ===
Saved: Trans_Z+_motion.gif

=== Running motion test: Rot_X+ ===
Saved: Rot_X+_motion.gif

=== Running motion test: Rot_Y+ ===
Saved: Rot_Y+_motion.gif

=== Running motion test: Rot_Z+ ===
Saved: Rot_Z+_motion.gif


### Q3-Q4 Setup

In [ ]:
''' Create Robot Instance'''
pandaUid = p.loadURDF(os.path.join(pybullet_data.getDataPath(), "franka_panda\\panda.urdf"),useFixedBase=True)
p.resetBasePositionAndOrientation(pandaUid, [0, 0, 0], [0, 0, 0, 1])
initialJointPosition = [0,-np.pi/4,np.pi/4,-np.pi/4,np.pi/4,np.pi/4,np.pi/4,0,0,0,0,0]
robot = eye_in_hand_robot(pandaUid,initialJointPosition)
p.stepSimulation() # need to do this to initialize robot

for ITER in range(200):
    p.stepSimulation()
    '''Q3b'''
    #object_center=[0.5+0.2*np.sin(np.pi/2+ITER/10), 0.5+0.2*np.sin(ITER/20), 0.01]
    #p.resetBasePositionAndOrientation(boxId,object_center,p.getQuaternionFromEuler(object_orientation))
 
    ''' Match Camera Pose to Robot End-Effector and Get Image'''
    # Get end-effector position and orientation
    cameraPosition, cameraOrientation = robot.get_ee_position()
    
    # Get image from camera at end-effector
    rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
    
    ''' Magical Computer Vision Algorithm that gets locations of objects in the image, as object_loc (Do Not Remove)'''
    viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
    object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,camera_width, camera_height)

    ''' Do some things here to get robot control'''
    # Get Image Jacobian
    imageJacobian = getImageJacobian(object_loc[0], object_loc[1], depth, camera_focal_depth, camera_width, camera_height)
    
    # Get Camera Control (desired camera motion)
    delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, imageJacobian)
    
    # Get Joint Control (convert camera motion to joint motion)
    new_jointPositions = findJointControl(robot, delta_X, delta_Omega)
    
    #set Next Joint Targets
    robot.set_joint_position(new_jointPositions)
    
    # show image
    cv2.imshow("depth", depth)
    cv2.imshow("rgb", rgb)
    cv2.waitKey(1)

#close the physics server
cv2.destroyAllWindows()    
p.disconnect() 